## Segmenting and Clustering Neighbourhoods in Toronto

### Part 1

In [2]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
import json 
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 


Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: | 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                              -                                      failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - cffi -> python[version='2.7.*|3.5.*|3.6.*|3.6.12|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.9,<3.10.0a0|>=3.8,<3.9.0a0|3.6.9|3.6.9|3.6.9|>=2.7,<2.8.0a0|3.6.9|>=3.5,<3.6.0a0|3.4.*',build='3_73_pypy|4_73_pypy|2_73_pypy|1_73_pypy|0_73_pypy']
  - geopy -> python[version='2.7.*|3.4.*|3.5.*|3.6.*']
  - rsa -> python[version='2.7.*|3.4.

In [4]:
import requests 
from pandas.io.json import json_normalize 

In [5]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [6]:
from sklearn.cluster import KMeans

In [7]:

!pip install folium
import folium 

print('Libraries imported.')

     |████████████████████████████████| 93 kB 4.0 MB/s  eta 0:00:01
Libraries imported.


#### Get the Table from the webpage.

In [8]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)
df_table= dfs[0]

#### Get the cells that have an assigned borough.

In [9]:
df= df_table[df_table['Borough']!='Not assigned'] 

#### Replace values where Neighbourhood is not assigned 

In [10]:
df.Neighbourhood.replace('Not assigned',df.Borough,inplace = True)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


#### Combine neighbourhoods with the same postal code and display the table.

In [11]:
df.groupby(['Postal Code']).agg(','.join)
df.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Get the number of rows of the dataframe.

In [12]:
df.shape

(103, 3)

### Part 2

#### Get the CSV file to creat a dataframe

In [13]:
url_2 = 'http://cocl.us/Geospatial_data'
df_2 = pd.read_csv(url_2)
df_2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
df_geo = pd.merge(df,df_2,on='Postal Code')
df_geo.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Part 3

#### Define Foursquare Credentials and Version

In [23]:
CLIENT_ID = 'V4MKX0ERS4ITCWSLGCGXZ2R105NC15JNWIKEH0COQBSP5OLW'
CLIENT_SECRET = 'LLMNRMPJBL31FTMP3GSMOG1AW2GK33TWXKJCC3DALCAL5RGB'
ACCESS_TOKEN = '2GKAWHI3WYR2Q4WMI253KXZJACJDB2PPPVCOTYDDGGSHDOUG'
VERSION = '20180605' 


print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: V4MKX0ERS4ITCWSLGCGXZ2R105NC15JNWIKEH0COQBSP5OLW
CLIENT_SECRET:LLMNRMPJBL31FTMP3GSMOG1AW2GK33TWXKJCC3DALCAL5RGB


#### Get the geographical coordinates of Toronto

In [20]:
address = 'Toronto,ON'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


#### Let's explore the Victoria Village in North York

#### Get the geographical coordinates of Victoria Village

In [32]:
neighbourhood_latitude= df_geo.loc[1,'Latitude']
neighbourhood_longitude= df_geo.loc[1, 'Longitude']
neighbourhood_name = df_geo.loc[1, 'Neighbourhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Victoria Village are 43.725882299999995, -79.31557159999998.


#### Let's get the top 100 venues that are in Victoria Village within a radius of 500 meters.

#### Define the  URL

In [35]:
LIMIT = 100 
radius = 500
URL = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
URL

'https://api.foursquare.com/v2/venues/explore?&client_id=V4MKX0ERS4ITCWSLGCGXZ2R105NC15JNWIKEH0COQBSP5OLW&client_secret=LLMNRMPJBL31FTMP3GSMOG1AW2GK33TWXKJCC3DALCAL5RGB&v=20180605&ll=43.725882299999995,-79.31557159999998&radius=500&limit=100'

#### Send the GET request and examine the resutls

In [36]:
results = requests.get(URL).json()
results

{'meta': {'code': 200, 'requestId': '5ff3fdfe6675562bfb2142d9'},
 'response': {'headerLocation': 'Bermondsey',
  'headerFullLocation': 'Bermondsey, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 6,
  'suggestedBounds': {'ne': {'lat': 43.7303823045, 'lng': -79.30935618239715},
   'sw': {'lat': 43.72138229549999, 'lng': -79.32178701760282}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c633acb86b6be9a61268e34',
       'name': 'Victoria Village Arena',
       'location': {'lat': 43.72348055545508,
        'lng': -79.31563520925143,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.72348055545508,
          'lng': -79.31563520925143}],
        'distance': 267,
        'cc': 'CA',
        'country': 'Canada',
        'formatte

#### Extract the category of the venue

In [37]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Convert the json file into dataframe

In [40]:
venues = results['response']['groups'][0]['items']
    
nearby_venues_VV = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_VV =nearby_venues_VV.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_VV['venue.categories'] = nearby_venues_VV.apply(get_category_type, axis=1)

# clean columns
nearby_venues_VV.columns = [col.split(".")[-1] for col in nearby_venues_VV.columns]

nearby_venues_VV.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Victoria Village Arena,Hockey Arena,43.723481,-79.315635
1,Portugril,Portuguese Restaurant,43.725819,-79.312785
2,Tim Hortons,Coffee Shop,43.725517,-79.313103
3,The Frig,French Restaurant,43.727051,-79.317418
4,Eglinton Ave E & Sloane Ave/Bermondsey Rd,Intersection,43.726086,-79.313620


In [41]:
print('{} venues were returned by Foursquare.'.format(nearby_venues_VV.shape[0])) 

6 venues were returned by Foursquare.


In [62]:
venues_map = folium.Map(location=[neighbourhood_latitude, neighbourhood_longitude], zoom_start=15) # generate map centred around the Victoria Village

for lat, lng, label in zip(nearby_venues_VV.lat, nearby_venues_VV.lng, nearby_venues_VV.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='white',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

### Let's explore Queen's Park in Downtown Toronto

In [54]:
#Get the geographical coordinates of Queen's Park
neighbourhood2_latitude= df_geo.loc[4,'Latitude']
neighbourhood2_longitude= df_geo.loc[4, 'Longitude']
neighbourhood2_name = df_geo.loc[4, 'Neighbourhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood2_name, 
                                                               neighbourhood2_latitude, 
                                                               neighbourhood2_longitude))

Latitude and longitude values of Queen's Park, Ontario Provincial Government are 43.6623015, -79.3894938.


In [55]:
#define URL and get the results
LIMIT = 100 
radius = 500
URL_2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood2_latitude, 
    neighbourhood2_longitude, 
    radius, 
    LIMIT)
results_2 = requests.get(URL_2).json()
results_2

{'meta': {'code': 200, 'requestId': '5ff40859a00d400630fc9c33'},
 'response': {'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 33,
  'suggestedBounds': {'ne': {'lat': 43.6668015045, 'lng': -79.38328496866619},
   'sw': {'lat': 43.657801495499996, 'lng': -79.3957026313338}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b9d206bf964a520e69136e3',
       'name': "Queen's Park",
       'location': {'address': 'University Ave.',
        'crossStreet': 'at Wellesley Ave.',
        'lat': 43.66394609897775,
        'lng': -79.39217952520835,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.66394609897775,
          'lng': -79.39217952520835}],


In [63]:
# Convert into dataframe
venues = results_2['response']['groups'][0]['items']
    
nearby_venues_QP = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_QP =nearby_venues_QP.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_QP['venue.categories'] = nearby_venues_QP.apply(get_category_type, axis=1)

# clean columns
nearby_venues_QP.columns = [col.split(".")[-1] for col in nearby_venues_QP.columns]

nearby_venues_QP.head(10)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Queen's Park,Park,43.663946,-79.392180
1,Mercatto,Italian Restaurant,43.660391,-79.387664
2,NEO COFFEE BAR,Coffee Shop,43.660130,-79.385830
3,Nando's,Portuguese Restaurant,43.661728,-79.386391
4,Bar Volo,Beer Bar,43.665462,-79.385692
5,T-Swirl Crepe,Creperie,43.663452,-79.384125
6,Starbucks,Coffee Shop,43.662407,-79.385943
7,The Yoga Sanctuary,Yoga Studio,43.661499,-79.383636
8,Central YMCA,Distribution Center,43.663083,-79.385025
9,Como En Casa,Mexican Restaurant,43.665160,-79.384796


In [57]:
print('{} venues were returned by Foursquare.'.format(nearby_venues_QP.shape[0])) 

33 venues were returned by Foursquare.


In [65]:
venues_map1 = folium.Map(location=[neighbourhood2_latitude, neighbourhood2_longitude], zoom_start=15) # generate map centred around the Queen's Park

for lat, lng, label in zip(nearby_venues_QP.lat, nearby_venues_QP.lng, nearby_venues_QP.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='white',
        fill_opacity=0.6
    ).add_to(venues_map1)

# display map
venues_map1

### By comparing the number of venues in Vic